In [9]:
import os

In [10]:
%pwd

'C:\\Users\\athar\\Projects\\Natural_Language_Processing\\Text-Summarizer-Project'

In [11]:
os.chdir("C:/Users/athar/Projects/Natural_Language_Processing/Text-Summarizer-Project")

In [12]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path 
    tokenizer_name: Path  
    

In [13]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    """
    A class to manage configuration settings for a project.
    This class handles loading YAML configuration files, creating necessary directories,
    and providing configuration objects for specific components of the project.
    """
    
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,  # Default path for the main configuration file
        params_filepath=PARAMS_FILE_PATH  # Default path for the parameters file
    ):
        """
        Initialize the ConfigurationManager instance.

        Args:
            config_filepath (str): Path to the main configuration file (YAML).
            params_filepath (str): Path to the parameters file (YAML).
        """
        # Load the YAML configuration files into Python dictionaries
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Log the loaded configuration for debugging purposes
        print("Loaded Config:", self.config)
        
        # Create the directory specified in the `artifacts_root` key of the configuration
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config= DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        
        return data_transformation_config

In [16]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [17]:
class DataTransformation:
    def __init__(self, config=DataTransformationConfig):
        # We are initializing this helper with some settings (config).
        # These settings tell us where the data is, and which tokenizer to use.
        self.config = config
        
        # We are using a special "tokenizer" to break text into smaller pieces (tokens).
        # It's like cutting big sentences into words or tiny understandable bits.
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

        
    def convert_examples_to_features(self, example_batch):
        # Here, we take one small group (batch) of data, with dialogues and summaries.

        # We break the dialogues into tokens (pieces of text) so the machine can understand it.
        # max_length=1024 means we don’t allow dialogues to be too long.
        input_encodings = self.tokenizer(
            example_batch['dialogue'], 
            max_length=1024, 
            truncation=True
        )
        
        # Now we break the summaries into tokens too. Summaries are shorter, so max_length=128.
        # We tell the tokenizer, "You are now cutting text for answers (targets)."
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'], 
                max_length=128, 
                truncation=True
            )
        
        # We return the "ingredients" needed for the model: input_ids, attention_masks, and labels.
        # - input_ids: Numbers that represent the words in the dialogue.
        # - attention_mask: Helps the machine know which words to focus on.
        # - labels: Numbers that represent the words in the summary.
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    
    
    def convert(self):
        # Load the entire dataset from the disk.
        # It’s like opening a box of data that has all the dialogues and summaries.
        dataset_samsum = load_from_disk(self.config.data_path)
        
        # Use the function above (convert_examples_to_features) to process each group of data.
        # 'map' means we apply the function to all the batches in the dataset.
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        
        # Save the processed data back to disk for later use.
        # We create a new folder inside the root directory called "samsum_dataset".
        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, "samsum_dataset")
        )


In [18]:
try:
    config=ConfigurationManager()
    data_transformation_config= config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-01-28 19:52:04,449: INFO: common: yaml file:config\config.yaml  loaded successfully]
[2025-01-28 19:52:04,451: INFO: common: yaml file:params.yaml  loaded successfully]
Loaded Config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}}
[2025-01-28 19:52:04,454: INFO: common: created directory at: artifacts]
[2025-01-28 19:52:04,456: INFO: common: created directory at: artifacts/data_transformation]


c:\Users\athar\Projects\Natural_Language_Processing\Text-Summarizer-Project\testS\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\athar\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Us